In [1]:
from agentic_blocks.utils.tools_utils import create_tool_registry, execute_and_add_tool_responses
from agentic_blocks.utils.tools_utils import execute_pending_tool_calls
from agentic_blocks.utils.tools_utils import print_tool
from agentic_blocks import call_llm, Messages
from deep_research.tools import internet_search, write_todos, think_tool
from IPython.display import display, Markdown

In [2]:
tools = [internet_search, write_todos, think_tool]

tool_registry = create_tool_registry(tools)

#print_tool(think_tool)

In [3]:
research_instructions = """You are an expert researcher. Your job is to conduct thorough research, and then write a polished report.

First use the write_todos tool to create a todo list.

When you have a list of todos, use the internet_search tool to conduct deep research. It will respond to your questions/topics with a detailed answer.

When you think you enough information to write a final report provide the final report.

Here are instructions for writing the final report:

<report_instructions>

CRITICAL: If you make a todo plan - you should note in the plan what language the report should be in so you dont forget!

Note: Write the report in English.

Please create a detailed answer to the overall research brief that:
1. Is well-organized with proper headings (# for title, ## for sections, ### for subsections)
2. Includes specific facts and insights from the research
3. References relevant sources using [Title](URL) format
4. Provides a balanced, thorough analysis. Be as comprehensive as possible, and include all information that is relevant to the overall research question. People are using you for deep research and will expect detailed, comprehensive answers.
5. Includes a "Sources" section at the end with all referenced links

You can structure your report in a number of different ways. Here are some examples:

To answer a question that asks you to compare two things, you might structure your report like this:
1/ intro
2/ overview of topic A
3/ overview of topic B
4/ comparison between A and B
5/ conclusion

To answer a question that asks you to return a list of things, you might only need a single section which is the entire list.
1/ list of things or table of things
Or, you could choose to make each item in the list a separate section in the report. When asked for lists, you don't need an introduction or conclusion.
1/ item 1
2/ item 2
3/ item 3

To answer a question that asks you to summarize a topic, give a report, or give an overview, you might structure your report like this:
1/ overview of topic
2/ concept 1
3/ concept 2
4/ concept 3
5/ conclusion

If you think you can answer the question with a single section, you can do that too!
1/ answer

REMEMBER: Section is a VERY fluid and loose concept. You can structure your report however you think is best, including in ways that are not listed above!
Make sure that your sections are cohesive, and make sense for the reader.

For each section of the report, do the following:
- Use simple, clear language
- Use ## for section title (Markdown format) for each section of the report
- Do NOT ever refer to yourself as the writer of the report. This should be a professional report without any self-referential language. 
- Do not say what you are doing in the report. Just write the report without any commentary from yourself.
- Each section should be as long as necessary to deeply answer the question with the information you have gathered. It is expected that sections will be fairly long and verbose. You are writing a deep research report, and users will expect a thorough answer.
- Use bullet points to list out information when appropriate, but by default, write in paragraph form.

REMEMBER:
The brief and research may be in English, but you need to translate this information to the right language when writing the final answer.
Make sure the final answer report is in English.

Format the report in clear markdown with proper structure and include source references where appropriate.

<Citation Rules>
- Assign each unique URL a single citation number in your text
- End with ### Sources that lists each source with corresponding numbers
- IMPORTANT: Number sources sequentially without gaps (1,2,3,4...) in the final list regardless of which sources you choose
- Each source should be a separate line item in a list, so that in markdown it is rendered as a list.
- Example format:
  [1] Source Title: URL
  [2] Source Title: URL
- Citations are extremely important. Make sure to include these, and pay a lot of attention to getting these right. Users will often use these citations to look into more information.
</Citation Rules>
</report_instructions>

You have access to these tools:

## `write_todos`

Use this to create and manage a structured task list. Pass an array of todo objects, each with:
- `content`: A string describing the task (required)
- `status`: One of "pending", "in_progress", or "completed" (optional, defaults to "pending")

Example: write_todos(todos=[{"content": "Research top restaurants", "status": "pending"}, {"content": "Write report", "status": "pending"}])

## `internet_search`

Use this to run an internet search for a given query. You can specify the number of results, the topic, and whether raw content should be included.

## `think_tool`

Use this to think about the research you have done so far. This will help you make decisions about what to do next.
"""

In [8]:
messages = Messages(
    system_prompt=research_instructions,
    user_prompt="Skriv en rapport om vad hände på Himmelska fridens torg 1989?"
)

#model = "qwen/qwen3-235b-a22b-2507"
#model = "qwen/qwen3-30b-a3b-thinking-2507"
model = "openai/gpt-oss-120b"

In [9]:
has_answer = False

while not has_answer:
    response = call_llm(model=model, messages=messages, tools=tools)
    messages.add_response_message(response)
    #print(messages.get_messages()[-1])
    
    if messages.has_pending_tool_calls():
        for tool_call in messages.get_pending_tool_calls():
            print(f"## Tool call: {tool_call['tool_name']}: {tool_call['arguments']}")
        #print(messages.get_pending_tool_calls())
        tool_responses = execute_pending_tool_calls(messages, tool_registry)
        messages.add_tool_responses(tool_responses)
    else:
        has_answer = True
    
    

## Tool call: write_todos: {'todos': [{'content': 'Research the background and causes of the Tiananmen Square protests in 1989.', 'status': 'pending'}, {'content': 'Gather detailed timeline of events at Tiananmen Square (Himmelska fridens torg) during April-June 1989.', 'status': 'pending'}, {'content': 'Find information on key figures, student leaders, and government officials involved.', 'status': 'pending'}, {'content': 'Collect data on the military crackdown on June 3-4, 1989, including casualties and international reactions.', 'status': 'pending'}, {'content': 'Research the aftermath and long-term impact of the Tiananmen Square events.', 'status': 'pending'}, {'content': 'Compile sources and citations for all gathered information.', 'status': 'pending'}, {'content': 'Write the final report in English with proper headings, citations, and a sources list.', 'status': 'pending'}]}
-------------- Writing todos --------------
- [pending] Research the background and causes of the Tiananm

LLMError: Failed to call LLM API: Error code: 429 - {'error': {'message': 'Provider returned error', 'code': 429, 'metadata': {'raw': 'openai/gpt-oss-120b is temporarily rate-limited upstream. Please retry shortly, or add your own key to accumulate your rate limits: https://openrouter.ai/settings/integrations', 'provider_name': 'Cerebras'}}, 'user_id': 'user_2vS6bJxDIasjEDBz8zqgs9VOObz'}

In [6]:
display(Markdown(response.content))

# The 1989 Tiananmen Square Protests: A Comprehensive Overview

## Overview of the Events

The 1989 Tiananmen Square protests were a series of pro-democracy demonstrations primarily led by students and workers in Beijing, China. What began as student-led marches in April 1989 evolved into the largest political protest in the history of communist China, with more than one million participants gathering in Tiananmen Square by mid-May [Tiananmen Square Protests: 1989, Massacre & Tank Man | HISTORY](https://www.history.com/articles/tiananmen-square). The demonstrations were motivated by a desire for democratic reforms, freedom of speech, freedom of the press, and an end to government corruption.

The protests gained momentum following the death of Hu Yaobang on April 15, 1989. Hu had been a reformist leader within the Chinese Communist Party and had been forced to resign in 1987 for his liberal views [Tiananmen Square incident | Massacre, Summary, Details, & Tank ...](https://www.britannica.com/event/Tiananmen-Square-incident). His passing sparked widespread mourning and provided a catalyst for the expression of broader political frustrations.

## Timeline of Key Events

### April 1989: The Protests Begin

- **April 15, 1989**: Former Communist Party leader Hu Yaobang passes away, triggering public mourning and unrest.
- **April 19**: Approximately 8,000 students marched to Zhongnanhai, the compound where top government leaders reside, to hold a sit-in protest demanding dialogue with authorities [A Brief Chronology of Events at Tiananmen Square (1989)](https://afe.easia.columbia.edu/special/china_1950_tiananmen_timeline.htm).
- **April 26**: The *People's Daily*, the official newspaper of the Communist Party, published an editorial denouncing the protests as an “organized conspiracy to sow chaos” [Timeline: Tiananmen protests - BBC News](https://www.bbc.com/news/world-asia-china-27404764). This escalated tensions, and over 150,000 students defiantly returned to Tiananmen Square, breaking through police lines.

### May 1989: Escalation and National Support

- **May 13**: In anticipation of Soviet leader Mikhail Gorbachev’s official visit to Beijing, about 160 students initiated an indefinite hunger strike in Tiananmen Square [Timeline: What Led to the Tiananmen Square Massacre - PBS](https://www.pbs.org/wgbh/frontline/article/timeline-tiananmen-square/). Their aim was to pressure the government into opening dialogue.
- The hunger strike galvanized widespread public support. Workers, intellectuals, and ordinary citizens joined the demonstrations.
- **May 17–18**: Close to one million people marched to the square, making it one of the largest political gatherings in China at the time [A Brief Chronology of Events at Tiananmen Square (1989)](https://afe.easia.columbia.edu/special/china_1950_tiananmen_timeline.htm).
- **May 19**: Demonstrators managed to prevent military units from entering the square, showcasing organized and strategic resistance.

### June 1989: The Crackdown

- **June 2**: The People’s Liberation Army (PLA), estimated at 300,000 troops, began moving into Beijing with tanks and armored personnel carriers to suppress the protests [Timeline: What Led to the Tiananmen Square Massacre - PBS](https://www.pbs.org/wgbh/frontline/article/timeline-tiananmen-square/).
- **June 3–4**: During the night and into the early morning, government forces violently cleared Tiananmen Square. Troops opened fire on demonstrators, crushed people with tanks, and made mass arrests. This period is often referred to as the Tiananmen Square Massacre and marked the deadliest instance of bloodshed in Beijing under communist rule [Timeline: Tiananmen protests - BBC News](https://www.bbc.com/news/world-asia-china-27404764).
- **June 4**: The square was entirely cleared of protesters after sustained use of military force.

## The Tank Man Incident

One of the most iconic and enduring images of the protests occurred on **June 5, 1989**, when an unidentified man stood alone in front of a column of Type 59 tanks as they moved west from Tiananmen Square. The man, later dubbed “Tank Man,” blocked their path multiple times, placing shopping bags on the lead tank’s barrel. The footage of this peaceful act of defiance was widely broadcast internationally, though censored in China. The fate of the man remains unknown, but the image has become a global symbol of resistance against authoritarian power [Tiananmen Square: What happened in the protests of 1989? - BBC](https://www.bbc.com/news/world-asia-48445934).

## Casualties and Suppression

The precise number of deaths and injuries remains highly controversial and is not officially disclosed. The Chinese government has never released a complete account of the events. However, reports from human rights organizations and foreign observers estimate that several hundred to thousands of unarmed protesters were killed during the military crackdown.

Amnesty International states that “the Chinese authorities want everyone to forget that they killed hundreds, if not thousands, of unarmed peaceful pro-democracy protesters” in Tiananmen Square [What really happened in the 1989 Tiananmen Square protests](https://www.amnesty.org.uk/china-1989-tiananmen-square-protests-demonstration-massacre). The suppression was not limited to Beijing; similar demonstrations in cities across China were also quelled in the following days.

## Political and Social Context

The 1989 protests emerged amid a broader movement for political reform in China. Inspired by global shifts toward democratization—most notably Mikhail Gorbachev’s liberalization policies in the Soviet Union—Chinese students and intellectuals sought greater freedoms and political transparency. Their demands included:

- An end to political corruption
- Reform of the monopoly on power held by the Communist Party
- Expansion of freedom of speech and press
- Protection of civil rights and democratic participation
- Dialogue between the government and the people

The Chinese leadership at the time, particularly Premier Li Peng and other hardliners, viewed the demonstrations as a threat to the socialist system and the Communist Party. Li Peng stated that the protests were being exploited by a “tiny minority” to subvert party leadership and socialism [Timeline: What Led to the Tiananmen Square Massacre - PBS](https://www.pbs.org/wgbh/frontline/article/timeline-tiananmen-square/).

## Aftermath and Global Response

The international community reacted with shock and condemnation. Western governments and media widely labeled the events a massacre and called for sanctions against China. Diplomatic relations were strained, and numerous countries imposed arms embargoes.

However, the Chinese government maintained strict censorship of all information related to the events. Public discussion of the 1989 demonstrations remains highly restricted in China. Terms such as "Tiananmen Square" and related keywords are heavily censored in digital spaces.

Despite decades of suppression, the events of 1989 continue to echo in global memory. Annual vigils are held around the world, particularly in Hong Kong (though increasingly restricted under new national security laws), and the story of the protests has been preserved by survivors, historians, and human rights advocates.

## Conclusion

The 1989 Tiananmen Square protests represent a pivotal moment in modern Chinese history. What began as a peaceful student-led movement for democratic reform culminated in a state-ordered military crackdown of unprecedented scale. The events underscore the tensions between political reform and authoritarian control during a period of rapid social and ideological change.

While official discourse in China remains silent, historical records, survivor accounts, and international media coverage affirm that thousands of civilians participated in peaceful demonstrations calling for democracy and human rights. The violent suppression that ended these protests had lasting consequences for China's political landscape and its relationship with the world.

The legacy of Tiananmen Square endures as a powerful reminder of the risks faced by citizens who demand transparency, freedom, and justice—values that continue to inspire pro-democracy movements globally.

### Sources

[1] Tiananmen Square: What happened in the protests of 1989? - BBC: https://www.bbc.com/news/world-asia-48445934  
[2] What really happened in the 1989 Tiananmen Square protests: https://www.amnesty.org.uk/china-1989-tiananmen-square-protests-demonstration-massacre  
[3] Tiananmen Square incident | Massacre, Summary, Details, & Tank ...: https://www.britannica.com/event/Tiananmen-Square-incident  
[4] 1989 Tiananmen Square massacre covered by CBS News - YouTube: https://www.youtube.com/watch?v=G7REeVXC_iM  
[5] What really happened in Tiananmen Square in 1989? - Reddit: https://www.reddit.com/r/DebateCommunism/comments/142boqy/what_really_happened_in_tiananmen_square_in_1989/  
[6] A Brief Chronology of Events at Tiananmen Square (1989): https://afe.easia.columbia.edu/special/china_1950_tiananmen_timeline.htm  
[7] Timeline: Tiananmen protests - BBC News: https://www.bbc.com/news/world-asia-china-27404764  
[8] Timeline: What Led to the Tiananmen Square Massacre - PBS: https://www.pbs.org/wgbh/frontline/article/timeline-tiananmen-square/  
[9] Tiananmen Square Protests: 1989, Massacre & Tank Man | HISTORY: https://www.history.com/articles/tiananmen-square  
[10] Causes and events of the Tiananmen Square protests - Fiveable: https://library.fiveable.me/history-modern-china/unit-16/events-tiananmen-square-protests/study-guide/xnJbYqXhvbHgTGYW

In [7]:
from markdown_pdf import MarkdownPdf, Section
from datetime import datetime

# Save the markdown content to PDF using markdown-pdf
def save_markdown_to_pdf(markdown_content, filename=None):
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"research_report_{timestamp}.pdf"
    
    try:
        # Create PDF with table of contents
        pdf = MarkdownPdf(toc_level=2, optimize=True)
        pdf.add_section(Section(markdown_content))
        pdf.save(filename)
        
        print(f"Report saved as PDF: {filename}")
    except Exception as e:
        print(f"Error saving PDF: {e}")
        print("Make sure to install: pip install markdown-pdf")

# Save the research report to PDF
if response.content and has_answer:  # Only save when we have the final answer
    save_markdown_to_pdf(response.content)

Report saved as PDF: research_report_20250902_065236.pdf
